In [96]:
#setup data
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing

#define risk free rate(rfr) as the three month treasure yield (DGS3MO)
rfr = np.array((5244,1))
#define term spread as 10 year treasure interest(DGS10) minus 1 year treasure interest(DGS1)
ts = np.array((5244,1))
#define default spread as the yield gap between AAA(DAAA) and BBB(DBAA) grade bonds
ds = np.array((5244,1))
ones = np.ones((5243,1))[:,0]

data1 = pd.read_csv('spread.csv')
data2 = pd.read_csv('StockMarket.csv')
data = pd.merge(data1, data2, on='Date')
rfr = data['DGS3MO']
ds = data['DGS10'] - data['DGS1']
ts = data['DBAA'] - data['DAAA']
# print(rfr.shape)
# print(ds.shape)
# print(ts.shape)
# print(ones.shape)
# A = np.stack((rfr, ds, ts, ones), axis = 1)

rfr2=np.zeros((5243,))
ds2=np.zeros((5243,))
ts2=np.zeros((5243,))
for i in range(5243):
    rfr2[i] = rfr[i+1]-rfr[i]
    ds2[i] = ds[i+1]-ds[i]
    ts2[i] = ts[i+1]-ts[i]
#delete the last data since there is no label for the next day
# print(data)
print(rfr[1:].shape)
print(rfr2.shape)
# A = np.stack((rfr[1:], ds[1:], ts[1:], rfr2, ds2, ts2, ones), axis = 1)
# A = np.stack((rfr[1:], ds[1:], rfr2, ds2, ones), axis = 1)
A = np.stack((rfr2, ds2, ones), axis = 1)
A = preprocessing.normalize(A)
# print(A)

#the yield of three major index: Dow Jones, SP 500 and Nasdaq
B_raw = np.stack((data['DJIA'], data['SP500'], data['NASDAQ']), axis = 1)
#Labeled as -1 if the market loss the next day, +1 otherwise 
B = np.zeros((5244, 3))
for i in range(5244):
    for j in range(3):
        B[i][j] = -1 if B_raw[i][j] < 0 else 1

#delete first label since there is no previous day data to predict
B= np.delete(B, 0, 0)

(5243,)
(5243,)


In [97]:
#LASSO, this code is copied from previous assignment
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X


In [98]:
##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,525],[526,1050],[1051,1575],[1576,2100],[2101,2625],[2626,3150],[3151,3675],[3676,4200],[4201,4725],[4726,5243]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

lam_vals = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
er_sq = np.zeros((cases,1))
er_rate = np.zeros((cases,1))

In [99]:
# DJIA
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]
    
    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,0].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,0], 2) / len(Bv2[:,0])
    error_vec2 = [0 if Bv2[j][0]==Bp2[j] else 1 for j in range(len(Bv2[:,0]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,0])

[1.95356324 4.06004369 0.08123785]
[1.81652791 3.6712168  0.07585151]
[2.13534749 3.80486971 0.05462977]
[0.90336723 4.37888398 0.06193807]
[1.0485373  3.84035909 0.06112346]
[1.78085581 2.53813315 0.05944936]
[1.40246635 2.64622993 0.04590272]
[2.23350748 3.80485162 0.06030751]


In [100]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.05777429]
 [0.05764856]
 [0.05950666]
 [0.05839889]
 [0.04996144]
 [0.05237229]
 [0.05405405]
 [0.06224813]]
Average square error: 
 0.056495539937201314
Error rate for each case: 
 [[0.43809524]
 [0.43619048]
 [0.4647619 ]
 [0.44761905]
 [0.32761905]
 [0.36      ]
 [0.37837838]
 [0.50857143]]
Average error rate: 
 0.42015444015444015


In [101]:
# SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,1].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,1], 2) / len(Bv2[:,1])
    error_vec2 = [0 if Bv2[j][1]==Bp2[j] else 1 for j in range(len(Bv2[:,1]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,1])

[1.67698337 3.72928124 0.09612741]
[1.56048302 3.47254275 0.08189298]
[1.84963727 3.48350112 0.05550419]
[0.89188378 4.14594751 0.06866539]
[0.69836631 3.38558364 0.06764031]
[1.52499829 2.20715359 0.06412665]
[1.77041637 2.98144011 0.05962289]
[2.08302218 3.56935317 0.07127559]


In [102]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.05999244]
 [0.05777429]
 [0.06059419]
 [0.05789975]
 [0.04981599]
 [0.05278631]
 [0.05405405]
 [0.06294366]]
Average square error: 
 0.056982585063097516
Error rate for each case: 
 [[0.47238095]
 [0.43809524]
 [0.48190476]
 [0.44      ]
 [0.32571429]
 [0.36571429]
 [0.37837838]
 [0.52      ]]
Average error rate: 
 0.4277734877734878


In [103]:
# NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,2].reshape((Bt.shape[0],1)),lam_vals)
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,2], 2) / len(Bv2[:,2])
    error_vec2 = [0 if Bv2[j][2]==Bp2[j] else 1 for j in range(len(Bv2[:,2]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,2])

[1.44024845 3.5715792  0.1018637 ]
[1.25963726 3.16314032 0.10167824]
[1.52353403 3.15749921 0.07904288]
[0.57242747 3.57047501 0.08659574]
[1.02852173 3.31716642 0.08431269]
[1.41704577 2.3699467  0.07837683]
[1.6105171  2.97487304 0.07174972]
[1.70735835 3.34336699 0.08250459]


In [104]:
print("The classifier based on Nasdaq index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Nasdaq index:
Square error for each case: 
 [[0.05962848]
 [0.05889381]
 [0.05739626]
 [0.05999244]
 [0.05039526]
 [0.05181513]
 [0.05514617]
 [0.05999244]]
Average square error: 
 0.05665749849804731
Error rate for each case: 
 [[0.46666667]
 [0.4552381 ]
 [0.43238095]
 [0.47238095]
 [0.33333333]
 [0.35238095]
 [0.39382239]
 [0.47238095]]
Average error rate: 
 0.42232303732303733


In [79]:
#Ridge Regression
#Dow Jones
for j in range(cases):
        # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,0]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,0]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,0]
    

    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)


[0.46689343 1.31847654 0.07957822]
[0.48858595 1.19451478 0.07519014]
[0.5988402  1.27552674 0.05417907]
[0.20038707 1.42494811 0.06180407]
[0.19995349 1.09838793 0.05975037]
[0.55780379 0.81603234 0.05868655]
[0.57854434 1.07607224 0.04784361]
[0.58009167 1.26453622 0.05954353]


In [80]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06224813]
 [0.05987137]
 [0.05777429]
 [0.06119001]
 [0.05223356]
 [0.05827451]
 [0.05648162]
 [0.06201455]]
Average square error: 
 0.058761004958174934
Error rate for each case: 
 [[0.50857143]
 [0.47047619]
 [0.43809524]
 [0.49142857]
 [0.35809524]
 [0.44571429]
 [0.41312741]
 [0.5047619 ]]
Average error rate: 
 0.4537837837837838


In [81]:
#SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,1]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,1]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,1]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[0.4458261  1.26812514 0.09617861]
[0.44530333 1.16483021 0.08278775]
[0.56732771 1.22186161 0.05654869]
[0.17003587 1.33108537 0.06701887]
[0.13189526 0.99498607 0.06640711]
[0.48851271 0.72725998 0.06341715]
[0.52671526 1.02788959 0.0592192 ]
[0.54661336 1.19378623 0.07042513]


In [82]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06363159]
 [0.05739626]
 [0.05789975]
 [0.06130848]
 [0.05251066]
 [0.05975005]
 [0.05739795]
 [0.06271268]]
Average square error: 
 0.059075927833162886
Error rate for each case: 
 [[0.53142857]
 [0.43238095]
 [0.44      ]
 [0.49333333]
 [0.36190476]
 [0.46857143]
 [0.42664093]
 [0.51619048]]
Average error rate: 
 0.4588063063063063


In [85]:
#NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,2]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,2]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,2]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[0.36621741 1.21245693 0.10179983]
[0.40507956 1.1141733  0.10276342]
[0.49583012 1.14722908 0.0802939 ]
[0.2200933  1.25438527 0.08699723]
[0.20975213 0.9631225  0.08302756]
[0.43966864 0.78594898 0.07764523]
[0.47294022 1.03708861 0.07102376]
[0.44416935 1.14196862 0.08120916]


In [86]:
print("The classifier based on NASDAQ index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on NASDAQ index:
Square error for each case: 
 [[0.06282828]
 [0.05987137]
 [0.05714286]
 [0.06107131]
 [0.05427736]
 [0.05827451]
 [0.05778621]
 [0.06035421]]
Average square error: 
 0.058950762807092466
Error rate for each case: 
 [[0.51809524]
 [0.47047619]
 [0.42857143]
 [0.48952381]
 [0.38666667]
 [0.44571429]
 [0.43243243]
 [0.47809524]]
Average error rate: 
 0.4561969111969112
